BERT MODULE

In [ ]:
%%capture
!pip install torch

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
!jupyter notebook restart

[C 02:33:53.608 NotebookApp] No such file or directory: /content/restart


In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
from transformers import BertForSequenceClassification

In [ ]:
!rm -rf bert-base-cased

In [ ]:
!pip install transformers --upgrade




In [ ]:
train_datapath = f'train.csv'
df_train = pd.read_csv(train_datapath, dtype=str)


val_datapath = f'validate.csv'
df_val = pd.read_csv(val_datapath, dtype=str)

test_datapath = f'test.csv'
df_test = pd.read_csv(test_datapath, dtype=str)


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
labels = {'0':0,
          '1':1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label']]
        self.texts = [tokenizer(text,
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')


    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)

        return pooled_output


In [ ]:
def trainBert(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()
    cls_embeddings_with_text_train = []
    cls_embeddings_with_text_val = []
    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)

                labels = [label for label in train_label]
                decoded_texts = [tokenizer.decode(ids,skip_special_tokens=True) for ids in input_id]

                for text,embedding,label in zip(decoded_texts,output,labels):
                  cls_embeddings_with_text_train.append((text,embedding,label))

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)
                    labels = [label for label in val_label]
                    decoded_texts = [tokenizer.decode(ids,skip_special_tokens=True) for ids in input_id]

                    for text,embedding,label in zip(decoded_texts,output,labels):
                      cls_embeddings_with_text_val.append((text,embedding,label))

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
    return cls_embeddings_with_text_train,cls_embeddings_with_text_val

In [ ]:
def evaluateBert(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    cls_embeddings_with_text = []
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              decoded_texts = [tokenizer.decode(ids,skip_special_tokens=True) for ids in input_id]

              for text,embedding in zip(decoded_texts,output):
                cls_embeddings_with_text.append((text,embedding))


              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc



    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    return cls_embeddings_with_text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
EPOCHS = 5
bertModel = BertClassifier()
LR = 1e-6

##TODO : play with LR, number of classes, why Linear ? bert pretrained models, batch size

#cls_embeddings_with_text_train is extracted from the train data
cls_embeddings_with_text_train,cls_embeddings_with_text_val = trainBert(bertModel, df_train, df_val, LR, EPOCHS)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

100%|██████████| 277/277 [01:00<00:00,  4.61it/s]


Epochs: 1 | Train Loss:  3.061 | Train Accuracy:  0.000 | Val Loss:  3.041 | Val Accuracy:  0.000


100%|██████████| 277/277 [01:03<00:00,  4.39it/s]


Epochs: 2 | Train Loss:  2.875 | Train Accuracy:  0.000 | Val Loss:  2.930 | Val Accuracy:  0.000


...

Epochs: 3 | Train Loss:  2.772 | Train Accuracy:  0.076 | Val Loss:  2.835 | Val Accuracy:  0.160


100%|██████████| 277/277 [01:02<00:00,  4.43it/s]


Epochs: 4 | Train Loss:  2.686 | Train Accuracy:  0.566 | Val Loss:  2.756 | Val Accuracy:  0.840


100%|██████████| 277/277 [01:02<00:00,  4.42it/s]


Epochs: 5 | Train Loss:  2.618 | Train Accuracy:  0.897 | Val Loss:  2.696 | Val Accuracy:  0.960


In [ ]:
cls_embeddings_with_text_test = evaluateBert(bertModel, df_test)

Test Accuracy:  0.917


UNIFIED TF_IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd

In [ ]:
## TODO : pretraitement et tokenization

In [ ]:
bagOfWordsDocs = []
for document in list(df_train['text']) :
    bagOfWordsDocs.append(document.split(' '))
uniqueWords = set([x for xs in bagOfWordsDocs for x in xs])


In [ ]:
vectorizer = TfidfVectorizer() #tokenization + preprocessing
vectors = vectorizer.fit_transform(list(df_train['text']))
feature_names = vectorizer.get_feature_names_out()
dense = vectors.todense()
denselist = dense.tolist()
tdfidf_df = pd.DataFrame(denselist, columns=feature_names)

In [ ]:
tfidfTokens = vectorizer.get_feature_names_out()

In [ ]:
tdfidf_df

,000,10,100,1000,13,1500,1608,1700,17th,18b,...,you,your,yourself,zero,zones,ªso,²but,µbut,ðÿ,ðÿœ
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.037730,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.032787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.110087,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.125351,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
548,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
549,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
550,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
551,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
unified_tfidf = {}

for column_name in tdfidf_df:
    #for index, row in df.iterrows():
    #print(df[column_name][df[column_name].idxmax()])
    unified_tfidf[column_name] = tdfidf_df[column_name][tdfidf_df[column_name].idxmax()]


In [ ]:
sum = sum(unified_tfidf.values())
sum

322.34070490073333

In [ ]:
unified_tfidf = {k: v / sum for k, v in unified_tfidf.items()}


CBOW

In [1]:
#TODO : verifier distribution IDF  - liste de mots avec idf et entropy apres tokenization ()
#calculer idf sur un ensemble plus grand tel que wikipedia
#travailler sur la tokenization : fait-elle le lien entre pluriels et singuliers etc (utiliser des stemmer - mapping entre cbow embedding et son stem )
#le score de classification est-il le meme avec ou sans la racine (essayer ca pour bert)
#

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Assuming df_train['text'] is your list of text documents

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# Tokenization + Preprocessing
vectors = vectorizer.fit_transform(df_train['text'])
feature_names = vectorizer.get_feature_names_out()

# Get IDF values
idf_values = vectorizer.idf_

# Get Document Frequency (DF) values
# DF is the total number of documents containing the term
df_values = vectors.astype(bool).sum(axis=0)

# Convert vectors to dense matrix
dense = vectors.todense()
denselist = dense.tolist()

# Create DataFrame with TF-IDF and DF values
tdfidf_df = pd.DataFrame(denselist, columns=feature_names)
tdfidf_df['IDF'] = idf_values
tdfidf_df['DF'] = df_values.tolist()[0]

# Print DataFrame
print(tdfidf_df)


NameError: name 'df_train' is not defined

In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# TODO : remove stopwords
train_corpus = list(df_train['text'])

# Tokenize and create vocabulary
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_corpus)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

In [ ]:
tokenscbow = word_index.keys()

In [ ]:
len(tokenscbow)
len(tfidfTokens)
set1 = set(tokenscbow)
set2 = set(tfidfTokens)

# Find strings that are not common between the two lists
not_common_strings = set1.symmetric_difference(set2)

# Convert the resulting set back to a list
not_common_strings_list = list(not_common_strings)


In [ ]:
import numpy as np

context_window = 10

def generate_data(corpus, context_window, tokenizer):
    sequences = tokenizer.texts_to_sequences(corpus)
    X, y = [], []
    for sequence in sequences:
        for i in range(context_window, len(sequence) - context_window):
            context = sequence[i - context_window : i] + sequence[i + 1 : i + context_window + 1]
            target = sequence[i]
            X.append(context)
            y.append(target)
    return np.array(X), np.array(y)

X_train, y_train = generate_data(train_corpus, context_window, tokenizer)

In [ ]:
embedding_dim = 768

cbowModel = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=context_window*2),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

cbowModel.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
epochs = 20
batch_size = 16

cbowModel.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

Epoch 1/20
10049/10049 [==============================] - 44s 4ms/step - loss: 4.0155 - accuracy: 0.2497
Epoch 2/20
10049/10049 [==============================] - 36s 4ms/step - loss: 2.2820 - accuracy: 0.5596
Epoch 3/20
10049/10049 [==============================] - 36s 4ms/step - loss: 1.5916 - accuracy: 0.6841
Epoch 4/20
10049/10049 [==============================] - 36s 4ms/step - loss: 1.1994 - accuracy: 0.7525
Epoch 5/20
10049/10049 [==============================] - 36s 4ms/step - loss: 0.9376 - accuracy: 0.8005
Epoch 6/20
10049/10049 [==============================] - 36s 4ms/step - loss: 0.7518 - accuracy: 0.8346
Epoch 7/20
10049/10049 [==============================] - 36s 4ms/step - loss: 0.6120 - accuracy: 0.8609
Epoch 8/20
10049/10049 [==============================] - 35s 4ms/step - loss: 0.5028 - accuracy: 0.8846
Epoch 9/20
10049/10049 [==============================] - 36s 4ms/step - loss: 0.4207 - accuracy: 0.9011
Epoch 10/20
10049/10049 [==============================

array([   1,  295,   64, ...,   14, 1827,    3])

In [ ]:

val_corpus = list(df_val['text'])
X_val, y_val = generate_data(val_corpus, context_window, tokenizer)
evaluation_results = cbowModel.evaluate(X_val, y_val)


loss = evaluation_results[0]

accuracy = evaluation_results[1]

# Print the evaluation results
print(f'Evaluation Loss: {loss}')
print(f'Evaluation Accuracy: {accuracy}')



221/221 [==============================] - 1s 2ms/step - loss: 2.5532 - accuracy: 0.7434
Evaluation Loss: 2.5531647205352783
Evaluation Accuracy: 0.7433503270149231


In [ ]:
#get embeddings :
cbowEmbeddings = cbowModel.layers[0].get_weights()[0]

In [ ]:
cbowEmbeddings.shape

(3021, 768)

In [ ]:
#soft_attention_embeddings is the tfdidf of a word multiplied by its cbow embedding
soft_attention_embeddings = {}

for word, idx in word_index.items() :
    if idx < cbowEmbeddings.shape[0] :
        if word in unified_tfidf.keys() :
          soft_attention_embeddings[word] = (unified_tfidf[word] * cbowEmbeddings[idx])

        #print(f"Word: {word}, Embeddings: {embeddings[idx]}")

Fusion module

In [ ]:
cls_embeddings_with_text_train[0][1].shape

torch.Size([768])

In [ ]:
def get_vzs(cls_embeddings_with_text) :
  #vzs_train is the list containing the vz of each input text - each vz corresponds to one input text found in cls_embeddings_with_text_train[x][0]
  vzs = []
  # e is one element of cls_embeddings_with_text_train
  for e in cls_embeddings_with_text :
      text = e[0]
      vh = np.zeros(768)
      for word in text.split() :
          if word in soft_attention_embeddings.keys() :
            vh += soft_attention_embeddings[word]
      vb = e[1].cpu()

      vzs.append((vh+vb.detach().numpy()))
  return vzs


In [ ]:
vzs_train = get_vzs(cls_embeddings_with_text_train)
vzs_val= get_vzs(cls_embeddings_with_text_val)
vzs_test = get_vzs(cls_embeddings_with_text_test)

FC

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
import torch.optim as optim

In [ ]:
class FullyConnectedNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FullyConnectedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(vzs_train, [x[2].detach().item() for x in cls_embeddings_with_text_train], test_size=0.2, random_state=42)

X_train, y_train = vzs_train, [x[2].detach().item() for x in cls_embeddings_with_text_train]
X_val, y_val = vzs_val, [x[2].detach().item() for x in cls_embeddings_with_text_val]
X_test= vzs_test


In [ ]:
# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Define the FullyConnectedNN model
input_size = 768  # Size of the BERT [CLS] embeddings
hidden_size = 128  # Example hidden size
output_size = 2  # Example output size
model = FullyConnectedNN(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Validation

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_preds = torch.argmax(val_outputs, dim=1)
        val_accuracy = (val_preds == y_val_tensor).float().mean()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy.item():.4f}")


print("Training complete.")

Epoch 1/10, Loss: 0.6981, Val Loss: 0.6734, Val Accuracy: 0.5280
Epoch 2/10, Loss: 0.6577, Val Loss: 0.6143, Val Accuracy: 0.7440
Epoch 3/10, Loss: 0.6186, Val Loss: 0.5836, Val Accuracy: 0.7600
Epoch 4/10, Loss: 0.5871, Val Loss: 0.5677, Val Accuracy: 0.7040
Epoch 5/10, Loss: 0.5612, Val Loss: 0.5455, Val Accuracy: 0.7120
Epoch 6/10, Loss: 0.5383, Val Loss: 0.5051, Val Accuracy: 0.8240
Epoch 7/10, Loss: 0.5104, Val Loss: 0.4722, Val Accuracy: 0.8560
Epoch 8/10, Loss: 0.4899, Val Loss: 0.4466, Val Accuracy: 0.8800
Epoch 9/10, Loss: 0.4705, Val Loss: 0.4255, Val Accuracy: 0.8800
Epoch 10/10, Loss: 0.4503, Val Loss: 0.4081, Val Accuracy: 0.8880
Training complete.


In [ ]:
# Convert NumPy arrays to PyTorch tensors
def predict_final(vz) :
  X_test_tensor = torch.tensor(vz, dtype=torch.float32)

# Perform inference
  model.eval()
  with torch.no_grad():
      test_outputs = model(X_test_tensor)
      test_preds = torch.argmax(test_outputs)

# Convert predictions tensor to NumPy array
  test_preds_array = test_preds.numpy()

# Now, test_preds_array contains the predicted class labels for the test data
  print("Predictions:", test_preds_array)